In [ ]:
#Inserted XML data to MongoDB(Air Quality Dataset)

# Reading air_quality
air_quality_url = "https://data.cityofnewyork.us/api/views/c3uy-2p5r/rows.xml?accessType=DOWNLOAD"
air_quality_response = urllib.request.urlopen(air_quality_url)
air_quality_xml = ET.parse(air_quality_response)
air_quality_xml_root = air_quality_xml.getroot()
air_quality_data = []

# inserting into mongoDB
air_quality_collection = project_db["air_quality"]
# deleting extising data
air_quality_collection.drop()

for row in air_quality_xml_root[0].findall('row'):
    air_quality_data_row = {}
    for elem in row.iter():
        if elem is not row:
            key = elem.tag.lower()
            value = elem.text
            #{"key":"value"}             
            air_quality_data_row[key] = value
    if len(air_quality_data_row) != 0:
        air_quality_collection.insert_one(air_quality_data_row)
print("Inserted air_quality document count: ", air_quality_collection.count_documents({}))

In [ ]:
#inserted values in air quality table
air_quality_collection = project_db["air_quality"]

cursor = conn.cursor()
for row in air_quality_collection.find():
    date = row["start_date"]
    cursor.execute("INSERT INTO air_quality (date_of_interest, data_value) VALUES (%s,%s)", 
                   (date, row["data_value"]))
cursor.close()
conn.commit()

In [ ]:
#Plots using seaborn library (2nd plot)
cursor = conn.cursor();
cursor.execute("""SELECT date_of_interest, avg(case_count) 
    FROM covid_cases  
    WHERE date_of_interest >= '2020-01-01' and date_of_interest < '2021-01-01'
    group by date_of_interest order by date_of_interest asc""")
covid_cases_data = cursor.fetchall()
cursor.close()


covid_cases_data_df = pd.DataFrame(data = covid_cases_data,
                                   columns=["Date",
                                            "Covid Case Count"])
fig, ax = plt.subplots(figsize = (12,6))

fig = sns.scatterplot(data = covid_cases_data_df, 
                   x="Date", 
                   y="Covid Case Count",
                   ax=ax
                  )

In [ ]:
#Plots using seaborn library (3rd plot)

cursor = conn.cursor();
query = """
SELECT date_of_interest, avg(data_value) 
    FROM air_quality
    WHERE date_of_interest >= '2020-01-01' and date_of_interest < '2022-01-01'
    GROUP BY  date_of_interest order by date_of_interest asc
"""
cursor.execute(query)
air_quality_data = cursor.fetchall()
cursor.close()


air_quality_data_modified = []
for x in range(12):
    date = datetime(2020, x+1, 1)
    if x < 6:
        key, value = air_quality_data[0]
        air_quality_data_modified.append((date, value))
    else:
        key, value = air_quality_data[1]
        air_quality_data_modified.append((date, value))

air_quality_data_df = pd.DataFrame(data = air_quality_data_modified,
                                   columns=["Date", "Air Quality Data Value"])
fig, ax = plt.subplots(figsize = (12,6))

fig = sns.barplot(data = air_quality_data_df, x="Date", y="Air Quality Data Value", ax=ax)

x_dates = air_quality_data_df['Date'].dt.strftime('%Y-%m-%d').sort_values().unique()
arr = ax.set_xticklabels(labels=x_dates, rotation=45, ha='right')